In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from glob import glob
from tqdm import tqdm
from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

from torch.utils.tensorboard import SummaryWriter

/home/john/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def RetinaTransform(image):
    # Bringing image to 0-1 range, #ATTENTION: CHECK IF ALL IMAGES min max is 0 and 255
    image = image / 255
    
    
    image = resize(image, (64,64),anti_aliasing=False) # resizing image because original does not fit in memory.
    
    
    # Bringing data to CHW format
    # N is a batch size, C denotes a number of channels, 
    # H is a height of input planes in pixels, and W is width in pixels.
    image = image.transpose([2,0,1])
    
    #Fixing dtype to avoid runtime error and save memory
    image = torch.tensor(image ,dtype=torch.float32)
    
    return image

In [4]:
# # Load dataset into memory
# cuda_image_array = []
# data_folder = "Evaluation_Set/Validation"
# label_path = "Evaluation_Set/RFMiD_Validation_Labels.csv"
# label_frame = pd.read_csv(label_path)
# local_transform = RetinaTransform

# for image_name in tqdm(glob(data_folder+"/*")):
#     image = io.imread(image_name)

#     label_frame_index = int(image_name.split("/")[-1].split(".")[0])
#     label = label_frame[label_frame["ID"]==label_frame_index]["Disease_Risk"].values[0]

#     image = local_transform(image)
#     label =  torch.tensor(label ,dtype=torch.float32)

#     cuda_image_array.append((image,label))

100%|█████████████████████████████████████████| 640/640 [01:18<00:00,  8.13it/s]


In [19]:
class CicikNet(nn.Module):
    def __init__(self):
        super(CicikNet, self).__init__()

        
        self.lrelu = nn.LeakyReLU(0.1)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1,padding=1)
        # self.conv2 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=1,padding=2)
        # self.conv3 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=10, stride=1,padding=5)


        # First fully connected layer
        self.fc1 = nn.Linear(64*64, 640)
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(640, 2)
        
        self.dpl = nn.Dropout(p=0.85)
        
        self.sm = nn.Softmax(dim = 1)
        
    def forward(self, x):
        x1 = torch.flatten(self.conv1(x),1) 
        x = torch.cat([x1],dim=1)
        
        x = self.dpl(x)
        
        x = self.fc1(x)
        x = self.lrelu(x)
        
        x = self.fc2(x)
        
        output = self.sm(x)
        return output

In [20]:
class RetinaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_indices, transform=None):
        
        self.data = [cuda_image_array[idx] for idx in data_indices]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [21]:
# def plot_grad_flow(named_parameters):
#     ave_grads = []
#     layers = []
#     for n, p in named_parameters:
#         if(p.requires_grad) and ("bias" not in n):
#             layers.append(n)
#             ave_grads.append(p.grad.abs().mean())
#     plt.plot(ave_grads, alpha=0.3, color="b")
#     plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
#     plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
#     plt.xlim(xmin=0, xmax=len(ave_grads))
#     plt.xlabel("Layers")
#     plt.ylabel("average gradient")
#     plt.title("Gradient flow")
#     plt.grid(True)

In [22]:

# x, y = torch.randn(10, 2), (torch.rand(10) > .5).long()
# loss = focal_loss(x, y)

In [23]:
EPOCHS = 100
# bceloss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5]).to(device))
# celoss = nn.CrossEntropyLoss(weight=torch.tensor([0.2,0.8]).to(device))

focal_loss = torch.hub.load(
'adeelh/pytorch-multi-class-focal-loss',
model='FocalLoss',
alpha=torch.tensor([0.8, 0.2]).to(device),
gamma=2,
reduction='mean',
force_reload=False
)

bauc = BinaryAUROC(thresholds=None)

writer = SummaryWriter('experiments/crossval_test_experiment')
# weight_decay =1e-4

Using cache found in /home/john/.cache/torch/hub/adeelh_pytorch-multi-class-focal-loss_master


In [24]:
# focal_loss(torch.tensor([[0.0,1.0]]).to(device),torch.tensor([0]).to(device))

In [25]:
# Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)

# print(input.shape)
# print(input,target)
# print(loss(input,target))
# output = loss(input, target)
# output.backward()
# # Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# output = loss(input, target)
# output.backward()

In [26]:
def model_train_loop(net,train_dataloader):
    
    running_loss = 0.0
    predictions = []
    ground_truth = []
    
    for idx, data in enumerate(train_dataloader, 0):
        # get the inputs
        # print(i,data)
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
    
    
        # plt.imshow(inputs[0].cpu().numpy().transpose([2,1,0]))
        # plt.show()
        
        
        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = focal_loss(outputs, labels.long())
        
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=100)  # Clipping to avoid vanishing gradient problem we saw.
        
        optimizer.step()
        running_loss += loss.item()
        
        predictions.append(outputs[:,1].flatten())
        ground_truth.append(labels.flatten())
        
    # net.to('cpu')
    # plot_grad_flow(net.named_parameters())
    # plt.show()
    # net.to(device)
    
    predictions=torch.cat(predictions)
    ground_truth = torch.cat(ground_truth)
    
    
    healthy_f1 = f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)
    rocauc = bauc(predictions,ground_truth)
    
    print('[%d, %5d] loss: %.3f, ROC: %.3f, F1_healthy: %.3f\n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))
    return running_loss, rocauc,healthy_f1

In [27]:
def model_test_loop(net, test_dataloader):
    
    with torch.no_grad():
        
        running_loss = 0.0
        test_predictions = []
        test_ground_truth = []
        
        for idx, data in enumerate(test_dataloader, 0):

            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = net(inputs)
            loss = focal_loss(outputs, labels.long())
            running_loss += loss.item()

            test_predictions.append(outputs[:,1].flatten())
            test_ground_truth.append(labels.flatten())

        
        
        test_predictions=torch.cat(test_predictions)
        test_ground_truth = torch.cat(test_ground_truth)


        healthy_f1 = f1_score((test_predictions>0.5).float().cpu(),test_ground_truth.cpu(),pos_label=0)
        rocauc = bauc(test_predictions,test_ground_truth)
        
        print('[%d, %5d] Validation loss: %.3f, ROC: %.3f, F1_healthy: %.3f \n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))
        
        return running_loss, rocauc,healthy_f1


In [28]:
# f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)

In [29]:
# print(len(train_dataloader))

In [30]:
# MODEL learns the trivial case of all 1s, which means it has capacity to learn:


    
# print('Finished Training')

In [31]:
# torch.tensor([[1,2],[3,4]])[:,0]

In [32]:
kf = KFold(n_splits=2)

fold_rocauc = []
fold_f1healthy = []
fold_loss = []
for idx, (train_index, test_index) in tqdm(enumerate(kf.split(cuda_image_array))):
    train_dataset = RetinaDataset(data_indices=train_index)
    test_dataset = RetinaDataset(data_indices=test_index)
    
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=0)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
    
    ciciknet = CicikNet().to(device)
    # optimizer = optim.SGD(ciciknet.parameters(), lr=0.001, momentum = 0.9) # Understand why ADAM does not work with batchsize = 1
    optimizer = optim.SGD(ciciknet.parameters(), lr=0.001, momentum = 0.9)
    
    for epoch in range(EPOCHS):  # loop over the dataset multiple times
        loss_train, rocauc_train, f1healthy_train= model_train_loop(ciciknet,train_dataloader)
        loss_test, rocauc_test, f1healthy_test = model_test_loop(ciciknet,test_dataloader)
        
        writer.add_scalars('training/val rocauc', {"train_rocauc":rocauc_train,"validation_rocauc":rocauc_test}, epoch +1)
        writer.add_scalars('training/val healthy_f1', {"f1healthy_train":f1healthy_train,"f1healthy_test":f1healthy_test}, epoch +1)
        writer.add_scalars('training/val loss', {"loss_train":loss_train,"loss_test":loss_test}, epoch +1)
        
        
    loss_, rocauc, f1healthy = model_test_loop(ciciknet,test_dataloader)
    
    fold_rocauc.append(rocauc)
    fold_f1healthy.append(f1healthy)
    fold_loss.append(loss_)
    
    break

0it [00:00, ?it/s]

[1,   320] loss: 19.058, ROC: 0.417, F1_healthy: 0.372

[1,   320] Validation loss: 17.222, ROC: 0.534, F1_healthy: 0.309 

[2,   320] loss: 18.899, ROC: 0.478, F1_healthy: 0.384

[2,   320] Validation loss: 17.175, ROC: 0.639, F1_healthy: 0.308 

[3,   320] loss: 18.815, ROC: 0.549, F1_healthy: 0.384

[3,   320] Validation loss: 17.189, ROC: 0.642, F1_healthy: 0.307 

[4,   320] loss: 18.842, ROC: 0.531, F1_healthy: 0.380

[4,   320] Validation loss: 17.219, ROC: 0.620, F1_healthy: 0.307 

[5,   320] loss: 18.754, ROC: 0.553, F1_healthy: 0.385

[5,   320] Validation loss: 17.196, ROC: 0.633, F1_healthy: 0.307 

[6,   320] loss: 18.744, ROC: 0.590, F1_healthy: 0.385

[6,   320] Validation loss: 17.155, ROC: 0.622, F1_healthy: 0.309 

[7,   320] loss: 18.696, ROC: 0.591, F1_healthy: 0.389

[7,   320] Validation loss: 17.159, ROC: 0.630, F1_healthy: 0.308 

[8,   320] loss: 18.685, ROC: 0.579, F1_healthy: 0.391

[8,   320] Validation loss: 17.092, ROC: 0.645, F1_healthy: 0.306 

[9,   32

0it [01:20, ?it/s]

[100,   320] Validation loss: 14.101, ROC: 0.785, F1_healthy: 0.466 



In [33]:
print(fold_rocauc)

[tensor(0.7846, device='cuda:0')]


In [34]:
print(fold_f1healthy)

[0.46590909090909094]
